## Set up matching DF's

In [1]:
import pandas as pd
# Set pandas to display all columns
pd.set_option('display.max_columns', None)

In [2]:
toto_raw = pd.read_csv('Data/scrapers/Toto/totoAllSports.csv', index_col=0).drop_duplicates()
kambi_raw = pd.read_csv('Data/scrapers/unibet/unibet_allSports_odds.csv', index_col=0).drop_duplicates()
# Apply the condition and update outcome_label
toto_raw.loc[(toto_raw['sport'] == 'Mixed Martial Arts') & (toto_raw['Market Name'] == 'Winaar Gevecht'), 'Market Name'] = 'Wedstrijdnotering'

In [58]:
set(kambi_raw['bet_offer_type_english_name'])

{'3-Way Handicap',
 'Asian Handicap',
 'Asian Over/Under',
 'Correct Score',
 'Double Chance',
 'HT/FT',
 'Handicap',
 'Head to Head',
 'Match',
 'Occurrence Method',
 'Odd/Even',
 'Over/Under',
 'Player Last Occurrence',
 'Player Occurrence Line',
 'Player Occurrence Number',
 'Winner',
 'Winning Margin',
 'Yes/No'}

In [31]:
#American Football, Basketbal, Boksen, Darten, Football, Tennis, Ijshockey, Mixed Martial Arts, Snooker, Tafeltennis, Waterpolo
set(toto_raw['sport'])

{'Badminton',
 'Bandy',
 'Basketbal',
 'Boksen',
 'Cricket',
 'Darten',
 'Floorball',
 'Football',
 'Futsal',
 'Golf',
 'Handbal',
 'IJshockey',
 'Mixed Martial Arts',
 'Motorsport',
 'Snooker',
 'Squash',
 'TOTO Specials',
 'Tafeltennis',
 'Tennis',
 'Virtual American Football',
 'Virtual Basketbal',
 'Virtual Cricket',
 'Virtual Darten',
 'Virtual Voetbal',
 'Voetbal',
 'Volleybal',
 'Waterpolo'}

Below is a translation of the abbreviations for the column outcome types into their likely meanings in the context of betting odds:

- `--`: Undefined or No Outcome (e.g., no odds available).
- `AG`: Anytime Goal Scorer (player to score at any time in the match).
- `CS`: Correct Score (predict the exact final score of the event).
- `DC`: Double Chance (cover two outcomes, e.g., Home Win or Draw).
- `DN`: Draw No Bet (stake refunded if the match ends in a draw).
- `FS`: First Scorer (player to score the first goal/point in the event).
- `H1`: First Half Result (outcome at the end of the first half).
- `H2`: Second Half Result (outcome in the second half only).
- `HF`: Half-Time/Full-Time (predict the result at both half-time and full-time).
- `HH`: Head-to-Head (comparison between two participants, e.g., players or teams).
- `HL`: Handicap Line (spread betting; adjust the line to level the playing field).
- `LS`: Last Scorer (player to score the last goal/point in the event).
- `MH`: Match Handicap (spread betting for the entire match).
- `MR`: Match Result (predict the overall winner or draw for the match).
- `OE`: Odd/Even (predict whether the total points/goals scored will be odd or even).
- `WH`: Winning Half (which half will have the higher score).
- `WM`: Winning Margin (predict the margin by which a team/player will win).

In [ ]:
# filter down to suitable betting opps
kambi_filtered = kambi_raw[kambi_raw['bet_offer_type_english_name'].isin(['Match', 'Odd/Even', 'Player Occurrence Line', 'Asian Over/Under', 'Over/Under', 'Handicap', 'Asian Handicap', 'Yes/No', 'Head to Head'])]
toto_filtered = toto_raw[toto_raw['Outcome Type'].isin(['--', 'DN', 'OE', 'HH', 'HL', 'AG'])]

kambi_filtered = kambi_raw
toto_filtered = toto_raw[toto_raw['Outcome Type'].isin(['DN', 'OE', 'HH', 'HL', 'AG'])]

In [47]:
import pandas as pd
from fuzzywuzzy import fuzz, process


# Step 1: Standardize Event Names with Fuzzy Matching
def match_event_names(kambi_events, toto_events):
    matched_events = {}
    for event in kambi_events:
        match, score = process.extractOne(event, toto_events, scorer=fuzz.token_sort_ratio)
        if score > 90:  # Adjust the threshold as necessary
            matched_events[event] = match
        else:
            matched_events[event] = event  # Fallback to original if no good match
    return matched_events

# Map matched events from kambi to toto
matched_events = match_event_names(kambi_filtered['event_name'].unique(), toto_filtered['Event Name'].unique())
kambi_filtered['standard_event_name'] = kambi_filtered['event_name'].map(matched_events)

# Reverse the matched events dictionary for use in the toto table
reversed_matched_events = {v: k for k, v in matched_events.items()}
toto_filtered['standard_event_name'] = toto_filtered['Event Name'].map(reversed_matched_events).fillna(toto_filtered['Event Name'])

# Step 2: Ensure Line Consistency
kambi_filtered['Line'] = kambi_filtered['line'] / 10000  # Adjust to decimal
# kambi['Line'] = kambi['Line'].fillna('Placeholder')
toto_filtered['Line'] = toto_filtered['Market Name'].str.extract(r'(\d+\.?\d*)').astype(float)
# toto['Line'] = toto['Line'].fillna('Placeholder')

# Step 3: Standardize Betting Types
bet_type_mapping = {
    'goals over/under': 'Total Goals',
    'Winnaar Gevecht': 'Wedstrijdnotering'
    # Add more mappings as necessary
}
kambi_filtered['standard_bet_type'] = kambi_filtered['criterion_label'].replace(bet_type_mapping)
toto_filtered['standard_bet_type'] = toto_filtered['Market Name'].replace(bet_type_mapping)

# Step 4
kambi_filtered['standard_outcome'] = kambi_filtered['outcome_label'].apply(
    lambda x: ' '.join(x.split(', ')[::-1]) if ',' in x else x
)

# Assign kambi 'outcome_label' directly to 'standard_outcome' as it's already in the desired format
toto_filtered['standard_outcome'] = toto_filtered['Outcome Name']

# Step 5: Merge DataFrames on Standardized Columns
merged_data = pd.merge(
    kambi_filtered,
    toto_filtered,
    left_on=['standard_event_name', 'standard_bet_type', 'standard_outcome', 'Line'],
    right_on=['standard_event_name', 'standard_bet_type', 'standard_outcome', 'Line'],
    suffixes=('_kambi', '_toto')
)

# Output merged results with odds side by side
print(merged_data[['event_name', 'Market Name', 'criterion_label', 'outcome_label', 'Line']])

                        event_name      Market Name    criterion_label  \
0     Richie Burnett vs Adam Mould        Wedstrijd          Wedstrijd   
1     Richie Burnett vs Adam Mould        Wedstrijd          Wedstrijd   
2     Richie Burnett vs Adam Mould        Wedstrijd          Wedstrijd   
3     Richie Burnett vs Adam Mould        Wedstrijd          Wedstrijd   
4   Tommy Morris vs Johann Brouwer        Wedstrijd          Wedstrijd   
..                             ...              ...                ...   
83         Bo Nickal vs Paul Craig  Winnaar Gevecht  Wedstrijdnotering   
84       Jon Jones vs Stipe Miocic  Winnaar Gevecht  Wedstrijdnotering   
85       Jon Jones vs Stipe Miocic  Winnaar Gevecht  Wedstrijdnotering   
86       Jon Jones vs Stipe Miocic  Winnaar Gevecht  Wedstrijdnotering   
87       Jon Jones vs Stipe Miocic  Winnaar Gevecht  Wedstrijdnotering   

      outcome_label  Line  
0   Burnett, Richie   NaN  
1   Burnett, Richie   NaN  
2       Mould, Adam   NaN  

/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_97584/2029359965.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toto_filtered['standard_event_name'] = toto_filtered['Event Name'].map(reversed_matched_events).fillna(toto_filtered['Event Name'])
/var/folders/sl/1m_7cj8j3dj7j13w2_rhnsvh0000gp/T/ipykernel_97584/2029359965.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toto_filtered['Line'] = toto_filtered['Market Name'].str.extract(r'(\d+\.?\d*)').astype(float)
/var/folders/sl/1m_7cj8j3dj

In [50]:
merged_data = merged_data[['event_id_toto', 'sport_toto','event_id_kambi', 'sport_kambi', 'standard_event_name', 'standard_bet_type', 'standard_outcome', 'Outcome Type', 'bet_offer_type_name','Line', 'odds', 'Odds (Decimal)']].drop_duplicates()
merged_data['odds'] = merged_data['odds'] / 1000

In [51]:
merged_data

,event_id_toto,sport_toto,event_id_kambi,sport_kambi,standard_event_name,standard_bet_type,standard_outcome,Outcome Type,bet_offer_type_name,Line,odds,Odds (Decimal)
0,5922618,Darten,1022055773,DARTS,Richie Burnett vs Adam Mould,Wedstrijd,Richie Burnett,HH,Wedstrijd,NaN,0.00167,1.55
2,5922618,Darten,1022055773,DARTS,Richie Burnett vs Adam Mould,Wedstrijd,Adam Mould,HH,Wedstrijd,NaN,0.00215,2.29
4,5922623,Darten,1022055775,DARTS,Tommy Morris vs Johann Brouwer,Wedstrijd,Tommy Morris,HH,Wedstrijd,NaN,0.00135,1.43
6,5922623,Darten,1022055775,DARTS,Tommy Morris vs Johann Brouwer,Wedstrijd,Johann Brouwer,HH,Wedstrijd,NaN,0.00300,2.65
8,5922628,Darten,1022055766,DARTS,Adam Mould vs Reece Robinson,Wedstrijd,Adam Mould,HH,Wedstrijd,NaN,0.00187,1.86
10,5922628,Darten,1022055766,DARTS,Adam Mould vs Reece Robinson,Wedstrijd,Reece Robinson,HH,Wedstrijd,NaN,0.00187,1.85
12,5922626,Darten,1022055749,DARTS,Johann Brouwer vs Richie Burnett,Wedstrijd,Johann Brouwer,HH,Wedstrijd,NaN,0.00220,2.28
14,5922626,Darten,1022055749,DARTS,Johann Brouwer vs Richie Burnett,Wedstrijd,Richie Burnett,HH,Wedstrijd,NaN,0.00164,1.56
16,5922625,Darten,1022055762,DARTS,Reece Robinson vs Johann Brouwer,Wedstrijd,Reece Robinson,HH,Wedstrijd,NaN,0.00140,1.48
18,5922625,Darten,1022055762,DARTS,Reece Robinson vs Johann Brouwer,Wedstrijd,Johann Brouwer,HH,Wedstrijd,NaN,0.00275,2.47


## Analyze results match

In [35]:
# Calculate the absolute difference between 'odds' and 'Odds (Decimal)'
merged_data['difference'] = abs(merged_data['odds'] - merged_data['Odds (Decimal)'])

# Only keep records where there are 2 outcomes
merged_data = merged_data.groupby(
    ['event_id_toto', 'sport_toto', 'event_id_kambi', 'sport_kambi', 'standard_event_name', 'standard_bet_type']
).filter(lambda x: len(x) == 2)

# Sort by the difference in descending order
top_differences = merged_data.sort_values(by='difference', ascending=False)

# Get the top records with the biggest differences
top_records = top_differences.head(20)

top_records

,event_id_toto,sport_toto,event_id_kambi,sport_kambi,standard_event_name,standard_bet_type,standard_outcome,Line,odds,Odds (Decimal),difference


In [36]:
# Split 'standard_event_name' into two parts
split_names = merged_data['standard_event_name'].str.split(' vs ', expand=True)

# Recode 'standard_outcome'
merged_data['standard_outcome'] = merged_data.apply(
    lambda row: '1' if row['standard_outcome'] == split_names.loc[row.name, 0] 
                else ('2' if row['standard_outcome'] == split_names.loc[row.name, 1] else row['standard_outcome']),
    axis=1
)

merged_data['standard_outcome'] = merged_data['standard_outcome'].astype(str)

ValueError: Columns must be same length as key

In [ ]:
merged_data[merged_data['standard_bet_type'] == 'Jannik Sinner Wint een Set']

,event_id_toto,sport_toto,event_id_kambi,sport_kambi,standard_event_name,standard_bet_type,standard_outcome,Line,odds,Odds (Decimal),difference
2,5944135,Tennis,1022071872,TENNIS,Jannik Sinner vs Taylor Fritz,Jannik Sinner Wint een Set,Ja,NaN,1.04,1.04,0.00
3,5944135,Tennis,1022071872,TENNIS,Jannik Sinner vs Taylor Fritz,Jannik Sinner Wint een Set,Nee,NaN,12.50,8.25,4.25


In [ ]:
## Get odds for single event on 1 record
# Filter groups with exactly two rows
filtered_df = merged_data.groupby(
    ['event_id_toto', 'sport_toto', 'event_id_kambi', 'sport_kambi', 'standard_event_name', 'standard_bet_type']
).filter(lambda x: len(x) == 2)

# Proceed with pivoting only if there are any valid groups left
if not filtered_df.empty:
    # Pivot the filtered DataFrame
    reshaped_df = filtered_df.pivot(
        index=['event_id_toto', 'sport_toto', 'event_id_kambi', 'sport_kambi', 'standard_event_name', 'standard_bet_type'],
        columns='standard_outcome',
        values=['Line', 'odds', 'Odds (Decimal)', 'difference']
    )

    # Flatten the multi-index columns for readability
    reshaped_df.columns = ['_'.join(col).strip() for col in reshaped_df.columns.values]

    # Reset index to turn multi-index into columns
    reshaped_df.reset_index(inplace=True)
else:
    reshaped_df = pd.DataFrame()  # Create an empty DataFrame if no groups qualify

reshaped_df

,event_id_toto,sport_toto,event_id_kambi,sport_kambi,standard_event_name,standard_bet_type,Line_1,Line_2,Line_Ja,Line_Nee,odds_1,odds_2,odds_Ja,odds_Nee,Odds (Decimal)_1,Odds (Decimal)_2,Odds (Decimal)_Ja,Odds (Decimal)_Nee,difference_1,difference_2,difference_Ja,difference_Nee
0,5468089,Mixed Martial Arts,1021633417,MARTIAL_ARTS,Jon Jones vs Stipe Miocic,Wedstrijdnotering,NaN,NaN,NaN,NaN,1.15,5.75,NaN,NaN,1.14,5.25,NaN,NaN,0.01,0.50,NaN,NaN
1,5699912,Voetbal,1020719873,FOOTBALL,San Marino vs Gibraltar,Beide Teams Scoren,NaN,NaN,NaN,NaN,NaN,NaN,2.35,1.53,NaN,NaN,2.33,1.54,NaN,NaN,0.02,0.01
2,5825551,Mixed Martial Arts,1021633419,MARTIAL_ARTS,Bo Nickal vs Paul Craig,Wedstrijdnotering,NaN,NaN,NaN,NaN,1.11,7.00,NaN,NaN,1.08,6.75,NaN,NaN,0.03,0.25,NaN,NaN
3,5848540,Voetbal,1021874423,FOOTBALL,Ecuador vs Bolivia,Beide Teams Scoren,NaN,NaN,NaN,NaN,NaN,NaN,2.08,1.65,NaN,NaN,2.32,1.54,NaN,NaN,0.24,0.11
4,5850619,Voetbal,1021874421,FOOTBALL,Uruguay vs Colombia,Beide Teams Scoren,NaN,NaN,NaN,NaN,NaN,NaN,2.17,1.60,NaN,NaN,2.12,1.64,NaN,NaN,0.05,0.04
5,5850658,Voetbal,1021874447,FOOTBALL,Bolivia vs Paraguay,Beide Teams Scoren,NaN,NaN,NaN,NaN,NaN,NaN,2.14,1.65,NaN,NaN,2.10,1.65,NaN,NaN,0.04,0.00
6,5850661,Voetbal,1021874448,FOOTBALL,Colombia vs Ecuador,Beide Teams Scoren,NaN,NaN,NaN,NaN,NaN,NaN,2.63,1.44,NaN,NaN,2.47,1.48,NaN,NaN,0.16,0.04
7,5865420,Mixed Martial Arts,1022019255,MARTIAL_ARTS,Chris Weidman vs Eryk Anders,Wedstrijdnotering,NaN,NaN,NaN,NaN,2.18,1.68,NaN,NaN,2.10,1.67,NaN,NaN,0.08,0.01,NaN,NaN
8,5880676,Mixed Martial Arts,1022019259,MARTIAL_ARTS,Jonathan Martinez vs Marcus McGhee,Wedstrijdnotering,NaN,NaN,NaN,NaN,2.10,1.74,NaN,NaN,2.10,1.67,NaN,NaN,0.00,0.07,NaN,NaN
9,5885309,Mixed Martial Arts,1022019261,MARTIAL_ARTS,Jim Miller vs Damon Jackson,Wedstrijdnotering,NaN,NaN,NaN,NaN,2.23,1.65,NaN,NaN,2.25,1.59,NaN,NaN,0.02,0.06,NaN,NaN


In [ ]:
reshaped_df

Line  \
standard_outcome                                                                                                                 Adam Mould   
event_id_toto sport_toto         event_id_kambi sport_kambi  standard_event_name                   standard_bet_type                          
5468088       Mixed Martial Arts 1021633418     MARTIAL_ARTS Charles Oliveira vs Michael Chandler  Wedstrijdnotering                    NaN   
5468089       Mixed Martial Arts 1021633417     MARTIAL_ARTS Jon Jones vs Stipe Miocic             Wedstrijdnotering                    NaN   
5699912       Voetbal            1020719873     FOOTBALL     San Marino vs Gibraltar               Beide Teams Scoren                   NaN   
5825551       Mixed Martial Arts 1021633419     MARTIAL_ARTS Bo Nickal vs Paul Craig               Wedstrijdnotering                    NaN   
5848540       Voetbal            1021874423     FOOTBALL     Ecuador vs Bolivia                    Beide Teams Scoren                   NaN   
5850619       Voetbal            1021874421     FOOTBALL     Uruguay vs Colombia                   Beide Teams Scoren                   NaN   
5850658       Voetbal            1021874447     FOOTBALL     Bolivia vs Paraguay                   Beide Teams Scoren                   NaN   
5850661       Voetbal            1021874448     FOOTBALL     Colombia vs Ecuador                   Beide Teams Scoren                   NaN   
5865420       Mixed Martial Arts 1022019255     MARTIAL_ARTS Chris Weidman vs Eryk Anders          Wedstrijdnotering                    NaN   
5880676       Mixed Martial Arts 1022019259     MARTIAL_ARTS Jonathan Martinez vs Marcus McGhee    Wedstrijdnotering                    NaN   
5885309       Mixed Martial Arts 1022019261     MARTIAL_ARTS Jim Miller vs Damon Jackson           Wedstrijdnotering                    NaN   
5887010       Mixed Martial Arts 1022019258     MARTIAL_ARTS Mickey Gall vs Ramiz Brahimaj         Wedstrijdnotering                    NaN   
5896181       Mixed Martial Arts 1022019257     MARTIAL_ARTS Mauricio Ruffy vs James Llontop       Wedstrijdnotering                    NaN   
5905931       Voetbal            1021999829     FOOTBALL     Angola vs Ghana                       Beide Teams Scoren                   NaN   
5905936       Voetbal            1021983458     FOOTBALL     Chad vs Sierra Leone                  Beide Teams Scoren                   NaN   
5905938       Voetbal            1021983457     FOOTBALL     Liberia vs Togo                       Beide Teams Scoren                   NaN   
5915669       Voetbal            1021991853     FOOTBALL     Benin vs Nigeria                      Beide Teams Scoren                   NaN   
5915694       Voetbal            1022007570     FOOTBALL     Ethiopia vs Tanzania                  Beide Teams Scoren                   NaN   
5915715       Voetbal            1022007578     FOOTBALL     Guinea vs DR Congo                    Beide Teams Scoren                   NaN   
5915736       Voetbal            1021999832     FOOTBALL     Eswatini vs Guinea-Bissau             Beide Teams Scoren                   NaN   
5915902       Voetbal            1021999831     FOOTBALL     Mozambique vs Mali                    Beide Teams Scoren                   NaN   
5915941       Voetbal            1021991850     FOOTBALL     Burkina Faso vs Senegal               Beide Teams Scoren                   NaN   
5915943       Voetbal            1021991848     FOOTBALL     Burundi vs Malawi                     Beide Teams Scoren                   NaN   
5922618       Darten             1022055773     DARTS        Richie Burnett vs Adam Mould          Wedstrijd                            NaN   
5922623       Darten             1022055775     DARTS        Tommy Morris vs Johann Brouwer        Wedstrijd                            NaN   
5922624       Darten             1022055765     DARTS        Tommy Morris vs Richie Burnett        Wedstrijd                       

In [ ]:
matched_events

{'Daniil Medvedev vs De Alex Minaur': 'Daniil Medvedev vs Alex De Minaur',
 'Jannik Sinner vs Taylor Fritz': 'Jannik Sinner vs Taylor Fritz',
 'Carlos Alcaraz vs Andrey Rublev': 'Carlos Alcaraz vs Andrey Rublev',
 'Alexander Zverev vs Casper Ruud': 'Alexander Zverev vs Casper Ruud',
 'Reece Robinson vs Sebastian Białecki': 'Reece Robinson vs Sebastian Bialecki',
 'Richie Burnett vs Adam Mould': 'Richie Burnett vs Adam Mould',
 'Tommy Morris vs Johann Brouwer': 'Tommy Morris vs Johann Brouwer',
 'Adam Mould vs Reece Robinson': 'Adam Mould vs Reece Robinson',
 'Sebastian Białecki vs Tommy Morris': 'Sebastian Bialecki vs Tommy Morris',
 'Johann Brouwer vs Richie Burnett': 'Johann Brouwer vs Richie Burnett',
 'Sebastian Białecki vs Adam Mould': 'Sebastian Bialecki vs Adam Mould',
 'Reece Robinson vs Johann Brouwer': 'Reece Robinson vs Johann Brouwer',
 'Tommy Morris vs Richie Burnett': 'Tommy Morris vs Richie Burnett',
 'Johann Brouwer vs Sebastian Białecki': 'Johann Brouwer vs Sebastian B

In [ ]:
# event name is key
# event keys: Wedstrijd = Wedstrijd, outcome = 1 vs 2
#             Over/Under criterion_label + line : 'Totaal Aantal Gewonnen Games door Alcaraz, Carlos' + 125000.0 vs Market Name: 'Carlos Alcaraz Aantal Games - Over/Under 12.5'
set(kambi_filtered[kambi_filtered['sport'] == 'TENNIS'][['criterion_label',
       'criterion_english_label', 'occurrence_type', 'lifetime',
       'bet_offer_type_id', 'bet_offer_type_name',
       'bet_offer_type_english_name', 'event_id', 'outcome_id',
       'outcome_label', 'outcome_english_label', 'odds', 'line', 'participant',
       'type', 'status',
       'cash_out_status', 'home_score', 'away_score', 'event_name']].criterion_label)

{'Alex Minaur Wint een Set',
 'Alexander Zverev Wint een Set',
 'Andrey Rublev Wint een Set',
 'Carlos Alcaraz Wint een Set',
 'Casper Ruud Wint een Set',
 'Correcte Score - Set 1',
 'Daniil Medvedev Wint een Set',
 'Game Handicap',
 'Game Handicap - Set 1',
 'Jannik Sinner Wint een Set',
 'Meeste aces',
 'Set Handicap',
 'Setwedden',
 'Taylor Fritz Wint een Set',
 'Totaal Aantal Games',
 'Totaal Aantal Games - Set 1',
 'Totaal Aantal Games - Set 2',
 'Totaal Aantal Gewonnen Games door Alcaraz, Carlos',
 'Totaal Aantal Gewonnen Games door De Minaur, Alex',
 'Totaal Aantal Gewonnen Games door Fritz, Taylor',
 'Totaal Aantal Gewonnen Games door Medvedev, Daniil',
 'Totaal Aantal Gewonnen Games door Rublev, Andrey',
 'Totaal Aantal Gewonnen Games door Ruud, Casper',
 'Totaal Aantal Gewonnen Games door Sinner, Jannik',
 'Totaal Aantal Gewonnen Games door Zverev, Alexander',
 'Totaal Aantal Sets',
 'Totaal Aantal Tiebreaks',
 'Totaal aantal minuten',
 'Totaal aantal servicebreaks',
 'Wedstr

In [ ]:
kambi_filtered[kambi_filtered['criterion_label']!='Wedstrijd'][['criterion_label',
       'criterion_english_label', 'occurrence_type', 'lifetime',
       'bet_offer_type_id', 'bet_offer_type_name',
       'bet_offer_type_english_name', 'event_id', 'outcome_id',
       'outcome_label', 'outcome_english_label', 'odds', 'line', 'participant',
       'type', 'status',
       'cash_out_status', 'home_score', 'away_score', 'event_name']].tail(60)

,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,status,cash_out_status,home_score,away_score,event_name
81408,Dubbele Kans - 1e Helft,Double Chance - 1st Half,GOALS,NaN,12,Dubbele kans,Double Chance,1021152946,3565547609,12,12,1640.0,NaN,NaN,OT_ONE_OR_TWO,OPEN,ENABLED,NaN,NaN,Port Vale vs Wrexham
81409,Dubbele Kans - 1e Helft,Double Chance - 1st Half,GOALS,NaN,12,Dubbele kans,Double Chance,1021152946,3565547615,X2,X2,1230.0,NaN,NaN,OT_CROSS_OR_TWO,OPEN,ENABLED,NaN,NaN,Port Vale vs Wrexham
81410,3-Way Handicap,3-Way Handicap,GOALS,FULL_TIME,11,3-Way handicap,3-Way Handicap,1021152946,3565547608,1,1,6750.0,-1000.0,Port Vale,1,OPEN,ENABLED,NaN,NaN,Port Vale vs Wrexham
81411,3-Way Handicap,3-Way Handicap,GOALS,FULL_TIME,11,3-Way handicap,3-Way Handicap,1021152946,3565547614,X,X,4700.0,-1000.0,NaN,OT_CROSS,OPEN,ENABLED,NaN,NaN,Port Vale vs Wrexham
81412,3-Way Handicap,3-Way Handicap,GOALS,FULL_TIME,11,3-Way handicap,3-Way Handicap,1021152946,3565547622,2,2,1290.0,-1000.0,Wrexham,2,OPEN,ENABLED,NaN,NaN,Port Vale vs Wrexham
81413,Beide Teams Scoren in Beide Helften,Both Teams to Score in Both Halves,NaN,NaN,18,Ja/Nee,Yes/No,1021152946,3565547638,Ja,Yes,15000.0,NaN,NaN,OT_YES,OPEN,ENABLED,NaN,NaN,Port Vale vs Wrexham
81414,Beide Teams Scoren in Beide Helften,Both Teams to Score in Both Halves,NaN,NaN,18,Ja/Nee,Yes/No,1021152946,3565547667,Nee,No,1020.0,NaN,NaN,OT_NO,OPEN,ENABLED,NaN,NaN,Port Vale vs Wrexham
81415,Correcte Score,Correct Score,GOALS,FULL_TIME,3,Correcte Score,Correct Score,1021152946,3565547619,0-0,0-0,9500.0,NaN,NaN,OT_UNTYPED,OPEN,ENABLED,0.0,0.0,Port Vale vs Wrexham
81416,Correcte Score,Correct Score,GOALS,FULL_TIME,3,Correcte Score,Correct Score,1021152946,3565547627,0-1,0-1,7500.0,NaN,NaN,OT_UNTYPED,OPEN,ENABLED,0.0,1.0,Port Vale vs Wrexham
81417,Correcte Score,Correct Score,GOALS,FULL_TIME,3,Correcte Score,Correct Score,1021152946,3565547635,0-2,0-2,10500.0,NaN,NaN,OT_UNTYPED,OPEN,ENABLED,0.0,2.0,Port Vale vs Wrexham


In [ ]:
toto_filtered[(toto_filtered['sport']=='Tennis') & (toto_filtered['event_id']==5943214)].drop_duplicates().head(50)

,event_id,Event Name,Market Name,Outcome Name,Odds (Decimal),Price Numerator,Price Denominator,Outcome Type,Outcome SubType,sport,competition
0,5943214,Daniil Medvedev vs Alex De Minaur,Alex De Minaur Wint een Set,Ja,1.35,7,20,--,NaN,Tennis,"ATP Finals, Enkelspel"
3,5943214,Daniil Medvedev vs Alex De Minaur,Alex De Minaur Wint een Set,Nee,2.95,39,20,--,NaN,Tennis,"ATP Finals, Enkelspel"
6,5943214,Daniil Medvedev vs Alex De Minaur,Wedstrijd,Alex De Minaur,1.93,93,100,HH,2,Tennis,"ATP Finals, Enkelspel"
9,5943214,Daniil Medvedev vs Alex De Minaur,Wedstrijd,Daniil Medvedev,1.92,23,25,HH,1,Tennis,"ATP Finals, Enkelspel"
12,5943214,Daniil Medvedev vs Alex De Minaur,Aantal Games - Odd/Even,Odd,1.87,87,100,OE,1,Tennis,"ATP Finals, Enkelspel"
15,5943214,Daniil Medvedev vs Alex De Minaur,Aantal Games - Odd/Even,Even,1.87,87,100,OE,2,Tennis,"ATP Finals, Enkelspel"
18,5943214,Daniil Medvedev vs Alex De Minaur,Set - Handicap 1.5,Alex De Minaur,2.95,39,20,WH,2,Tennis,"ATP Finals, Enkelspel"
21,5943214,Daniil Medvedev vs Alex De Minaur,Set - Handicap 1.5,Daniil Medvedev,1.35,7,20,WH,1,Tennis,"ATP Finals, Enkelspel"
24,5943214,Daniil Medvedev vs Alex De Minaur,Exact Sets,2 Sets,1.59,59,100,--,NaN,Tennis,"ATP Finals, Enkelspel"
27,5943214,Daniil Medvedev vs Alex De Minaur,Exact Sets,3 Sets,2.27,127,100,--,NaN,Tennis,"ATP Finals, Enkelspel"


In [ ]:
import pandas as pd
import re
from fuzzywuzzy import fuzz, process

# Function to normalize names
def normalize_name(name):
    name = re.sub(r'\b(de|het|een)\b', '', name, flags=re.IGNORECASE)  # Remove articles
    name = re.sub(r'\s+', ' ', name).strip()  # Remove extra spaces
    return name

# Function to match names using fuzzy matching
def match_names(kambi_names, toto_names):
    normalized_toto = {name: normalize_name(name) for name in toto_names}
    normalized_kambi = {name: normalize_name(name) for name in kambi_names}
    matched_names = {}
    for kambi_name, normalized_kambi_name in normalized_kambi.items():
        match, score = process.extractOne(normalized_kambi_name, normalized_toto.values(), scorer=fuzz.token_sort_ratio)
        if score > 85:  # Adjust threshold
            matched_names[kambi_name] = [k for k, v in normalized_toto.items() if v == match][0]
        else:
            matched_names[kambi_name] = kambi_name
    return matched_names

# Normalize and map names in Kambi and Toto
kambi_names = kambi_filtered['bet_offer_type_name'].unique()
toto_names = toto_filtered['Market Name'].unique()
matched_names = match_names(kambi_names, toto_names)
kambi_filtered['normalized_bet_offer_type'] = kambi_filtered['bet_offer_type_name'].map(matched_names)
toto_filtered['normalized_market_name'] = toto_filtered['Market Name'].apply(normalize_name)

# Merge DataFrames using normalized columns
merged_data = pd.merge(
    kambi_filtered,
    toto_filtered,
    left_on=['standard_event_name', 'normalized_bet_offer_type', 'standard_outcome', 'Line'],
    right_on=['standard_event_name', 'normalized_market_name', 'standard_outcome', 'Line'],
    suffixes=('_kambi', '_toto')
)

# Output merged results
print(merged_data[['event_name', 'Market Name', 'criterion_label', 'outcome_label', 'Line']].drop_duplicates())

                          event_name Market Name             criterion_label  \
0      Jannik Sinner vs Taylor Fritz   Wedstrijd           Wedstrijdnotering   
1      Jannik Sinner vs Taylor Fritz   Wedstrijd           Wedstrijdnotering   
2    Carlos Alcaraz vs Andrey Rublev   Wedstrijd           Wedstrijdnotering   
3    Carlos Alcaraz vs Andrey Rublev   Wedstrijd           Wedstrijdnotering   
4    Alexander Zverev vs Casper Ruud   Wedstrijd           Wedstrijdnotering   
..                               ...         ...                         ...   
236       Mark Selby vs Shaun Murphy   Wedstrijd                     Frame 1   
238       Mark Selby vs Shaun Murphy   Wedstrijd       Meeste Century Breaks   
240       Mark Selby vs Shaun Murphy   Wedstrijd       Meeste Century Breaks   
242       Mark Selby vs Shaun Murphy   Wedstrijd  Meeste Half-Century Breaks   
244       Mark Selby vs Shaun Murphy   Wedstrijd  Meeste Half-Century Breaks   

         outcome_label  Line  
0       

In [ ]:
merged_data[['event_name', 'Market Name', 'criterion_label', 'outcome_label', 'Line', 'odds', 'Odds (Decimal)','sport_toto']].drop_duplicates()

,event_name,Market Name,criterion_label,outcome_label,Line,odds,Odds (Decimal),sport_toto
0,Jannik Sinner vs Taylor Fritz,Wedstrijd,Wedstrijdnotering,"Sinner, Jannik",NaN,1150.0,1.15,Tennis
1,Jannik Sinner vs Taylor Fritz,Wedstrijd,Wedstrijdnotering,"Fritz, Taylor",NaN,5800.0,6.00,Tennis
2,Carlos Alcaraz vs Andrey Rublev,Wedstrijd,Wedstrijdnotering,"Alcaraz, Carlos",NaN,1340.0,1.30,Tennis
3,Carlos Alcaraz vs Andrey Rublev,Wedstrijd,Wedstrijdnotering,"Rublev, Andrey",NaN,3350.0,3.65,Tennis
4,Alexander Zverev vs Casper Ruud,Wedstrijd,Wedstrijdnotering,"Zverev, Alexander",NaN,1140.0,1.15,Tennis
...,...,...,...,...,...,...,...,...
236,Mark Selby vs Shaun Murphy,Wedstrijd,Frame 1,"Murphy, Shaun",NaN,1910.0,2.10,Snooker
238,Mark Selby vs Shaun Murphy,Wedstrijd,Meeste Century Breaks,"Selby, Mark",NaN,3000.0,1.73,Snooker
240,Mark Selby vs Shaun Murphy,Wedstrijd,Meeste Century Breaks,"Murphy, Shaun",NaN,3250.0,2.10,Snooker
242,Mark Selby vs Shaun Murphy,Wedstrijd,Meeste Half-Century Breaks,"Selby, Mark",NaN,2000.0,1.73,Snooker


In [ ]:
    left_on=['standard_event_name', 'normalized_bet_offer_type', 'standard_outcome', 'Line'],
    right_on['standard_event_name', 'normalized_market_name', 'standard_outcome', 'Line']=,

In [ ]:
toto_filtered[(toto_filtered['sport']=='Voetbal') & (toto_filtered['Event Name'].str.lower().str.contains('peru')) & (toto_filtered['normalized_market_name'].str.contains('Over/Under'))][['standard_event_name', 'normalized_market_name', 'standard_outcome', 'Line']]

,standard_event_name,normalized_market_name,standard_outcome,Line
14439,Peru vs Chile,Aantal Goals - Over/Under 2.5,Over,2.5
14441,Peru vs Chile,Aantal Goals - Over/Under 2.5,Under,2.5
14443,Peru vs Chile,Aantal Goals - Over/Under 3.5,Over,3.5
14445,Peru vs Chile,Aantal Goals - Over/Under 3.5,Under,3.5
14447,Peru vs Chile,Aantal Goals - Over/Under 0.5,Under,0.5
14449,Peru vs Chile,Aantal Goals - Over/Under 0.5,Over,0.5
14451,Peru vs Chile,Aantal Goals - Over/Under 1.5,Over,1.5
14453,Peru vs Chile,Aantal Goals - Over/Under 1.5,Under,1.5
14455,Peru vs Chile,Aantal Goals - Over/Under 4.5,Under,4.5
14457,Peru vs Chile,Aantal Goals - Over/Under 4.5,Over,4.5


In [ ]:
kambi_filterd[(kambi_filtered['sport']=='FOOTBALL') & kambi_filtered['event_name'].str.lower().str.contains('peru') & (kambi_filtered['normalized_bet_offer_type']=='Over/Onder')].head(60)

,bet_offer_id,criterion_id,criterion_label,criterion_english_label,occurrence_type,lifetime,bet_offer_type_id,bet_offer_type_name,bet_offer_type_english_name,event_id,outcome_id,outcome_label,outcome_english_label,odds,line,participant,type,changed_date,odds_fractional,odds_american,status,cash_out_status,home_score,away_score,event_name,sport,group_name,standard_event_name,Line,standard_bet_type,standard_outcome,normalized_bet_offer_type
67964,2470331702,1001159633,Totaal Aantal Doelpunten door Chili,Total Goals by Chile,NaN,FULL_TIME,6,Over/Onder,Over/Under,1021874459,3560403762,Meer dan,Over,3550.0,1500.0,NaN,OT_OVER,2024-11-11T12:45:13Z,5/2,255.0,OPEN,ENABLED,NaN,NaN,Peru vs Chile,FOOTBALL,NaN,Peru vs Chili,0.15,Totaal Aantal Doelpunten door Chili,Meer dan,Over/Onder
67965,2470331702,1001159633,Totaal Aantal Doelpunten door Chili,Total Goals by Chile,NaN,FULL_TIME,6,Over/Onder,Over/Under,1021874459,3560403766,Minder dan,Under,1250.0,1500.0,NaN,OT_UNDER,2024-11-11T12:45:13Z,1/4,-400.0,OPEN,ENABLED,NaN,NaN,Peru vs Chile,FOOTBALL,NaN,Peru vs Chili,0.15,Totaal Aantal Doelpunten door Chili,Minder dan,Over/Onder
67986,2470331715,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021874459,3560403804,Meer dan,Over,6000.0,3500.0,NaN,OT_OVER,2024-11-11T12:45:13Z,5/1,500.0,OPEN,ENABLED,NaN,NaN,Peru vs Chile,FOOTBALL,NaN,Peru vs Chili,0.35,Totaal Aantal Doelpunten,Meer dan,Over/Onder
67987,2470331715,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021874459,3560403806,Minder dan,Under,1130.0,3500.0,NaN,OT_UNDER,2024-11-11T12:45:13Z,1/8,-770.0,OPEN,ENABLED,NaN,NaN,Peru vs Chile,FOOTBALL,NaN,Peru vs Chili,0.35,Totaal Aantal Doelpunten,Minder dan,Over/Onder
67991,2470331718,1001159967,Totaal Aantal Doelpunten door Peru,Total Goals by Peru,NaN,FULL_TIME,6,Over/Onder,Over/Under,1021874459,3560403809,Meer dan,Over,2800.0,1500.0,NaN,OT_OVER,2024-11-11T12:45:13Z,9/5,180.0,OPEN,ENABLED,NaN,NaN,Peru vs Chile,FOOTBALL,NaN,Peru vs Chili,0.15,Totaal Aantal Doelpunten door Peru,Meer dan,Over/Onder
67992,2470331718,1001159967,Totaal Aantal Doelpunten door Peru,Total Goals by Peru,NaN,FULL_TIME,6,Over/Onder,Over/Under,1021874459,3560403812,Minder dan,Under,1380.0,1500.0,NaN,OT_UNDER,2024-11-11T12:45:13Z,4/11,-265.0,OPEN,ENABLED,NaN,NaN,Peru vs Chile,FOOTBALL,NaN,Peru vs Chili,0.15,Totaal Aantal Doelpunten door Peru,Minder dan,Over/Onder
68006,2470331724,1001159532,Totaal Aantal Doelpunten - 1e Helft,Total Goals - 1st Half,GOALS,NaN,6,Over/Onder,Over/Under,1021874459,3560403827,Meer dan,Over,1560.0,500.0,NaN,OT_OVER,2024-11-11T12:45:13Z,11/20,-180.0,OPEN,ENABLED,NaN,NaN,Peru vs Chile,FOOTBALL,NaN,Peru vs Chili,0.05,Totaal Aantal Doelpunten - 1e Helft,Meer dan,Over/Onder
68007,2470331724,1001159532,Totaal Aantal Doelpunten - 1e Helft,Total Goals - 1st Half,GOALS,NaN,6,Over/Onder,Over/Under,1021874459,3560403833,Minder dan,Under,2230.0,500.0,NaN,OT_UNDER,2024-11-11T12:45:13Z,6/5,123.0,OPEN,ENABLED,NaN,NaN,Peru vs Chile,FOOTBALL,NaN,Peru vs Chili,0.05,Totaal Aantal Doelpunten - 1e Helft,Minder dan,Over/Onder
68008,2470331725,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021874459,3560403835,Meer dan,Over,2750.0,2500.0,NaN,OT_OVER,2024-11-11T12:45:13Z,7/4,175.0,OPEN,ENABLED,NaN,NaN,Peru vs Chile,FOOTBALL,NaN,Peru vs Chili,0.25,Totaal Aantal Doelpunten,Meer dan,Over/Onder
68009,2470331725,1001159926,Totaal Aantal Doelpunten,Total Goals,GOALS,FULL_TIME,6,Over/Onder,Over/Under,1021874459,3560403842,Minder dan,Under,1450.0,2500.0,NaN,OT_UNDER,2024-11-11T12:45:13Z,4/9,-225.0,OPEN,ENABLED,NaN,NaN,Peru vs Chile,FOOTBALL,NaN,Peru vs Chili,0.25,Totaal Aantal Doelpunten,Minder dan,Over/Onder


In [ ]:
# toto: kambi
voetbal_map = {'Resultaat': 'Wedstrijd',
               'Aantal Goals - Over/Under 0.5': 'Totaal Aantal Doelpunten' #icm line & kambi['outcome_english_label'] = 'Over' &
               } 